In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd "drive/My Drive/MachineLearningPractice/tools"

/content/drive/My Drive/MachineLearningPractice/tools


In [3]:
!pip install import-ipynb
!pip3 install torch torchvision

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=37050a3adec689d834bd7bbac1c83611eb8391f7cec03e4a49c8f27edb89ffa4
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [4]:
import sys
sys.path.append('/content/drive/My Drive/MachineLearningPractice')

In [5]:
import torch.optim as optim
import torch.utils.data
import torchvision
import torch.nn as nn
import numpy as np
import argparse
import import_ipynb

In [6]:
from models.vgg import *
from models.alexnet import *
from tools.utils import _initialize_weights
from tools.utils import LoadData
# import tool.colab_init

importing Jupyter notebook from /content/drive/My Drive/MachineLearningPractice/models/vgg.ipynb
importing Jupyter notebook from /content/drive/My Drive/MachineLearningPractice/models/alexnet.ipynb
importing Jupyter notebook from /content/drive/My Drive/MachineLearningPractice/tools/utils.ipynb


In [7]:
class Solver(object):
  def __init__(self,config):
    self.ModelType = config.model
    self.model = None
    self.lr = config.lr
    self.epochs = config.epoch
    self.device = None
    self.DataType = "Cifar10"
    # self.train_batch_size = config.trainBatchSize
    # self.test_batch_size = config.testBatchSize
    self.criterion = None
    self.optimizer = None
    self.scheduler = None
  
  def DataLoading(self):
    dataset = LoadData(self.DataType)
    self.TrainSet = dataset['trainset']
    self.TestSet = dataset['testset']
    self.classes = dataset['classes']

  def ModelLoading(self):
    self.model = eval(self.ModelType)().to(self.device)
    _initialize_weights(self.model)
    self.optimizer = optim.SGD(self.model.parameters(),lr=self.lr,momentum=0.9,weight_decay=0.0006)
    # self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
    #scheduler
    self.criterion = nn.CrossEntropyLoss().to(self.device)
    
  def train(self,epoch):
    total_loss = 0
    total_correct = 0
    total_num = 0
    
    # kbar = pkbar.Kbar(target=len(self.TrainSet), epoch=epoch, num_epochs=self.epochs+1, width=10, always_stateful=False)
    for batch_num, (data, target) in enumerate(self.TrainSet):
      data, target = data.to(self.device), target.to(self.device)
      self.optimizer.zero_grad()
      output = self.model(data)
      loss = self.criterion(output, target)
      loss.backward()
      self.optimizer.step()
      total_loss += loss.item()
      prediction = torch.max(output, 1) #return a tuple
      total_num += target.size(0)
      total_correct += torch.sum((prediction[1] == target).int()).numpy()

      # kbar.update(batch_num,values=[("Loss",total_loss/(batch_num+1)),("Current Loss",loss.item()),("Acc",total_correct/total_num),("Correct",total_correct),("Sum",total_num)])
      if (batch_num%100==0):
          print("epoch:{:d} batch:{:d} loss:{:.4f} acc:{:.4f}({:d}/{:d})".format(epoch,batch_num,total_loss/(batch_num+1),total_correct/total_num,total_correct,total_num))
    return total_loss, total_correct/total_num

  def test(self,epoch):
    test_loss = 0
    test_correct = 0
    test_num = 0

    # kbar = pkbar.Kbar(target=len(self.TestSet), epoch=epoch, num_epochs=self.epochs+1, width=10, always_stateful=False)

    with torch.no_grad():
      for batch_num, (data, target) in enumerate(self.TestSet):
        data, target = data.to(self.device), target.to(self.device)
        output = self.model(data)
        loss = self.criterion(output, target)
        test_loss += loss.item()
        prediction = torch.max(output, 1) #return a tuple
        test_num += target.size(0)
        test_correct += torch.sum((prediction[1] == target).int()).numpy()

        # kbar.update(batch_num,values=[("Loss",test_loss/(batch_num+1)),("Acc",test_correct/test_num),("Correct",test_correct),("Sum",test_num)])
        if (batch_num%100==0):
          print("epoch:{:d} batch:{:d} loss:{:.4f} acc:{:.4f}({:d}/{:d})".format(epoch,batch_num,test_loss/(batch_num+1),test_correct/test_num,test_correct,test_num))
    return test_loss, test_correct/test_num

  def save(self):
    model_path = "model.pth"
    torch.save(self.model,model_path)
    print("Saved in {}".format(model_path))

  # load model data??

  def run(self):
    self.DataLoading()
    self.ModelLoading()
    accuracy = 0
    for epoch in range(1,self.epochs+1):
      # scheduler
      train_result = self.train(epoch)
      print(train_result)
      test_result = self.test(epoch)
      best_accuracy = max(accuracy,test_result[1])
      if epoch == self.epochs:
        print("\n==>Best Performance: %.3f%%"%(accuracy*100))
        self.save()

In [8]:
def main():
  parser = argparse.ArgumentParser(description="cifar-10 with PyTorch")
  parser.add_argument('--model',default='VGG19',type=str,help='model type for training')
  parser.add_argument('--lr',default=0.005,type=float,help='learning rate')
  parser.add_argument('--epoch',default=3,type=int,help="epochs for training")
  parser.add_argument('--trainBatchSize',default=100,type=int,help="batch size for training")
  parser.add_argument('--testBatchSize',default=100,type=int,help="batche size for test")
  parser.add_argument('--cuda',default=torch.cuda.is_available(),type=bool,help='whether cuda is in use')
  parser.add_argument('-f')
  args = parser.parse_args()

  solver = Solver(args)
  solver.run()

In [9]:
main()

Files already downloaded and verified
Files already downloaded and verified


/content/drive/My Drive/MachineLearningPractice/tools/utils.ipynb:4: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
/content/drive/My Drive/MachineLearningPractice/tools/utils.ipynb:11: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.


epoch:1 batch:0 loss:2.3024 acc:0.2500(1/4)
epoch:1 batch:100 loss:2.3023 acc:0.1015(41/404)
epoch:1 batch:200 loss:2.3045 acc:0.1020(82/804)
epoch:1 batch:300 loss:2.3014 acc:0.1013(122/1204)
epoch:1 batch:400 loss:2.2950 acc:0.1116(179/1604)
epoch:1 batch:500 loss:2.2948 acc:0.1173(235/2004)
epoch:1 batch:600 loss:2.2961 acc:0.1119(269/2404)
epoch:1 batch:700 loss:2.2977 acc:0.1106(310/2804)
epoch:1 batch:800 loss:2.2987 acc:0.1099(352/3204)
epoch:1 batch:900 loss:2.2988 acc:0.1107(399/3604)
epoch:1 batch:1000 loss:2.2998 acc:0.1106(443/4004)
epoch:1 batch:1100 loss:2.3005 acc:0.1104(486/4404)
epoch:1 batch:1200 loss:2.3012 acc:0.1087(522/4804)
epoch:1 batch:1300 loss:2.3017 acc:0.1078(561/5204)
epoch:1 batch:1400 loss:2.3021 acc:0.1074(602/5604)
epoch:1 batch:1500 loss:2.3023 acc:0.1073(644/6004)
epoch:1 batch:1600 loss:2.3026 acc:0.1056(676/6404)
epoch:1 batch:1700 loss:2.3024 acc:0.1051(715/6804)
epoch:1 batch:1800 loss:2.3025 acc:0.1051(757/7204)
epoch:1 batch:1900 loss:2.3029 ac

KeyboardInterrupt: ignored